### 0. Import Library

In [2]:
import pandas as pd, numpy as np
import pymysql, os, gspread

from tqdm import tqdm
from datetime import datetime

pd.set_option('display.max_columns', None)

def live_db_conn():
    conn = pymysql.connect(host='db-6j3k3.pub-cdb.ntruss.com', user='redtable', password='fpemxpdlqmf5491!@#',autocommit=True,cursorclass=pymysql.cursors.DictCursor, db = "redtable2021")
    return conn

### 1. Load Data

In [3]:
sa = gspread.service_account("../gspread_token/snappy-cosine-411501-fbfbf5c109c9.json")

iaurora = sa.open("레드테이블x아이오로라")
df_iaurora = iaurora.worksheet("외부인원_NEW_결제내역")
values = df_iaurora.get_all_values()
header, rows = values[0], values[1:]
df_iaurora = pd.DataFrame(rows, columns=header)

kona = sa.open("코나아이X코리아트래블카드")
df_kona = kona.worksheet("결제내역(코나)")
values = df_kona.get_all_values()
header, rows = values[1], values[2:]
df_kona = pd.DataFrame(rows, columns=header)
df_kona.rename(columns={'address':'주소'}, inplace=True)

def filter_and_assign_promotion(df, address_starts, promotion_name):
    condition = df["주소"].str.startswith(tuple(address_starts))
    columns = ["id", "masking_card_no", "16자리 카드번호", "사업자번호", "결제날짜", 
               "결제시간", "시/도", "시/군/구", "카테고리_대분류", "category_one", 
               "category_two", "상점명", "주소", "결제금액", "결제후잔액",
               "채널_대분류", "분출채널", "iaurora_id"]
    filtered_df = df.loc[condition, columns]
    filtered_df["프로모션"] = promotion_name
    filtered_df["카드형태"] = "아이오로라"
    filtered_df = filtered_df[filtered_df["결제날짜"] > "20240421"]
    return filtered_df

def kona_filter_and_assign_promotion(df, address_starts, promotion_name):
    condition = df["주소"].str.startswith(tuple(address_starts))
    df["masking_card_no"] = df["카드번호"]
    columns = ["id", "masking_card_no", "카드번호", "사업자번호", "결제날짜", 
               "결제시간", "시/도", "시/군/구", "카테고리_대분류", "category_one", 
               "category_two", "상점명", "주소", "결제금액", "결제후금액",
               "채널_대분류", "분출채널", "userId"]
    filtered_df = df.loc[condition, columns]
    filtered_df["프로모션"] = promotion_name
    filtered_df["카드형태"] = "코나"
    filtered_df = filtered_df[filtered_df["결제날짜"] > "20240421"]
    return filtered_df

promotions = {
    "롯데백화점 본점": ["서울특별시 중구 소공동 1", "서울특별시 중구 을지로1가 132-2", "서울특별시 중구 남대문로 81"],
    "롯데백화점 영플라자": ["서울특별시 중구 소공동 123", "서울특별시 중구 남대문로 67"],
    "롯데백화점 잠실점": ["서울특별시 송파구 잠실동 40-1", "서울특별시 송파구 올림픽로 240" , "서울특별시 송파구 송파대로 521"],
    "롯데백화점 캐슬플라자": ["서울특별시 송파구 신천동 7-18", "서울특별시 송파구 올림픽로 269"]
}

def df_preprocessing(df, index):
    # 각 프로모션에 대해 데이터 프레임 생성 및 병합
    df_list = []
    if index == "kona":
        for promotion, addresses in promotions.items():
            df_promotion = kona_filter_and_assign_promotion(df, addresses, promotion)
            df_list.append(df_promotion)
    else:
    
        for promotion, addresses in promotions.items():
            df_promotion = filter_and_assign_promotion(df, addresses, promotion)
            df_list.append(df_promotion)

    # 데이터 프레임 병합
    df_merge = pd.concat(df_list)

    # 중복 제거 및 정렬
    df_merge = df_merge.astype({"id":"int"})
    df_merge.drop_duplicates(subset=["id"], inplace=True)
    df_merge.sort_values("id", ascending=True, inplace=True)
    df_merge.reset_index(drop=True, inplace=True)
    return df_merge


df_merge_iaurora = df_preprocessing(df_iaurora, "iaurora")
df_merge_kona = df_preprocessing(df_kona, "kona")

df_merge_iaurora = df_merge_iaurora.rename(columns={"결제후잔액":"결제후금액", "iaurora_id":"card_id"})
df_merge_kona = df_merge_kona.rename(columns={"카드번호":"16자리 카드번호", "userId":"card_id"})

df_final = pd.concat([df_merge_iaurora, df_merge_kona], axis=0)
df_final.sort_values(by=["결제날짜", "결제시간"], ascending=True, inplace=True)

df_final.reset_index(drop=True, inplace=True)

df_final["결제금액"] = df_final["결제금액"].apply(lambda x: int(str(x).replace("₩","").replace(",","")))
df_final["결제후금액"] = df_final["결제후금액"].apply(lambda x: int(str(x).replace("₩","").replace(",","")))

def business_check(input_text):
    """롯데백화점 4개"""
    if str(input_text) in ("1048126067", "2018513497", "2198500066", "2158524595"):
        return "O"
    return "X"

df_final["롯데사업자_유무"] = df_final["사업자번호"].apply(lambda x:business_check(x))

df_final = df_final[['프로모션', '카드형태', 'id', 'masking_card_no', '16자리 카드번호',
                    '사업자번호', '롯데사업자_유무', '결제날짜', '결제시간', '시/도', 
                    '시/군/구', '카테고리_대분류', 'category_one', 'category_two', '상점명', 
                    '주소', '결제금액', '결제후금액', '채널_대분류', '분출채널', 'card_id']]


In [4]:
df = df_final

df.head()

,프로모션,카드형태,id,masking_card_no,16자리 카드번호,사업자번호,롯데사업자_유무,결제날짜,결제시간,시/도,시/군/구,카테고리_대분류,category_one,category_two,상점명,주소,결제금액,결제후금액,채널_대분류,분출채널,card_id
0,롯데백화점 본점,코나,28,9410411100080104,9410411100080104,1048126067,O,20240423,155934,서울특별시,중구,쇼핑,"쇼핑,유통",백화점,롯데백화점 본점,서울특별시 중구 소공동 1,105000,145503,내부인원,내부인원,3101644687
1,롯데백화점 본점,코나,50,9410411100080104,9410411100080104,1048126067,O,20240423,161047,서울특별시,중구,쇼핑,"쇼핑,유통",백화점,롯데백화점 본점,서울특별시 중구 소공동 1,97300,48413,내부인원,내부인원,3101644687
2,롯데백화점 본점,코나,339,9410411100080104,9410411100080104,1048126067,O,20240423,161047,서울특별시,중구,쇼핑,"쇼핑,유통",백화점,롯데백화점 본점,서울특별시 중구 소공동 1,97300,48413,내부인원,내부인원,3101644687
3,롯데백화점 본점,아이오로라,43524,9491********2329,9491********2329,1048126067,O,20240424,113518,서울특별시,중구,쇼핑,"쇼핑,유통",백화점,롯데쇼핑(명동본점),서울특별시 중구 소공동 1,4700,118163,기타,추정치,2001204000457533
4,롯데백화점 본점,아이오로라,43558,9491********0355,9491300045580355,1048126067,O,20240424,125033,서울특별시,중구,쇼핑,"쇼핑,유통",백화점,롯데쇼핑(명동본점),서울특별시 중구 소공동 1,42000,165413,내부인원,내부인원,2001203010028795


In [5]:
df[df['16자리 카드번호'] == ""]

,프로모션,카드형태,id,masking_card_no,16자리 카드번호,사업자번호,롯데사업자_유무,결제날짜,결제시간,시/도,시/군/구,카테고리_대분류,category_one,category_two,상점명,주소,결제금액,결제후금액,채널_대분류,분출채널,card_id


In [6]:
# df = pd.read_csv('../data/코나아이X코리아트래블카드 - 롯데백화점_결제내역(코나+IA).csv')

# df.head()

In [7]:
# df_kona = pd.read_csv('../data/코나아이X코리아트래블카드 - 결제내역(코나).csv', header=1)
# df_iaurora = pd.read_csv('../data/레드테이블x아이오로라 - 외부인원_NEW_결제내역.csv')

# df_kona = df_kona[['id', '카드번호', '사업자번호', '결제날짜', '결제시간', '시/도', '시/군/구', '카테고리_대분류',
#        'category_one', 'category_two', '상점명', '주소', '결제금액', '결제후금액','userId']]

# df_iaurora = df_iaurora[['id','16자리 카드번호',
#        '사업자번호', '결제날짜', '결제시간', '시/도', '시/군/구', '카테고리_대분류', 'category_one',
#        'category_two', '상점명', '주소', 'ktc_가맹여부', 'store_id', '결제금액',
#        '결제후잔액', 'user_id']]

# df_iaurora.rename(columns={'16자리 카드번호':'카드번호'}, inplace=True)

# KNdf_iaurora = pd.concat([df_kona, df_iaurora], axis=0)
# KNdf_iaurora = KNdf_iaurora[KNdf_iaurora['결제날짜'] != ' ']
# KNdf_iaurora['결제날짜'] = KNdf_iaurora['결제날짜'].apply(lambda x:int(x))
# KNdf_iaurora.head()

### 2. Preprocessing

In [8]:
# 영플라자 => 본점 취급
df['프로모션'] = df['프로모션'].apply(lambda x:x.replace('영플라자', '본점')) 

### 2. Set Condition

In [9]:
df['결제날짜'] = df['결제날짜'].apply(lambda x:pd.to_datetime(x))

In [10]:
# 프로모션 기간 설정
promotion_start_period = '2024-04-22'
promotion_end_period = '2024-05-10'

# 사업자 번호, 지점 확인 불가능 오류
error_store = ['산시 도삭면', '옵스', 'tamburins']

condition_1 = df['결제날짜'] >= promotion_start_period
condition_2 = df['결제날짜'] <= promotion_end_period
condition_3 = df['상점명'].isin(error_store)

In [11]:
target_df = df[(condition_1) & (condition_2) & ~(condition_3)]

### 3. Dashboard

In [12]:
len(target_df)

118

In [13]:
# 구매 고객 수 (A)
Promotion_Customer = target_df.drop_duplicates(subset=['결제날짜','card_id'])['프로모션'].value_counts()

Lotte_Myeongdong_BuyCustomer = Promotion_Customer[0]
Lotte_Jamsil_BuyCustomer= Promotion_Customer[1]

print('구매고객수')

print('본점:',Lotte_Myeongdong_BuyCustomer)

print('잠실점:',Lotte_Jamsil_BuyCustomer)

구매고객수
본점: 50
잠실점: 13


In [14]:
# 본점 - 캐시백 대상자, 이벤트 참여율
Event_Rate = target_df.groupby(['결제날짜','card_id'])[['결제금액']].sum().reset_index()

cashback_target = Event_Rate[Event_Rate['결제금액'] >= 200000]

print('전체')

print('캐시백 대상자:',len(cashback_target))

print('이벤트 참여율:',round(len(cashback_target)/(Lotte_Myeongdong_BuyCustomer + Lotte_Jamsil_BuyCustomer)*100, 2))

전체
캐시백 대상자: 32
이벤트 참여율: 50.79


In [15]:
# 본점 - 캐시백 대상자, 이벤트 참여율
Lotte_Myeongdong_df = target_df[target_df['프로모션'] == '롯데백화점 본점']

Event_Rate = Lotte_Myeongdong_df.groupby(['결제날짜','card_id'])[['결제금액']].sum().reset_index()

LM_cashback_target = Event_Rate[Event_Rate['결제금액'] >= 200000]

print('본점')

print('캐시백 대상자 수:',len(LM_cashback_target))

print('이벤트 참여율:',round(len(LM_cashback_target)/Lotte_Myeongdong_BuyCustomer*100, 2))

LM_cashback_won = len(LM_cashback_target) * 20000

print('캐시백 제공액:',LM_cashback_won)

본점
캐시백 대상자 수: 26
이벤트 참여율: 52.0
캐시백 제공액: 520000


In [16]:
# 잠실점 - 캐시백 대상자, 이벤트 참여율
Lotte_Jamsil_df = target_df[target_df['프로모션'] == '롯데백화점 잠실점']

Event_Rate = Lotte_Jamsil_df.groupby(['결제날짜','card_id'])[['결제금액']].sum().reset_index()

LJ_cashback_target = Event_Rate[Event_Rate['결제금액'] >= 200000]

print('잠실점')

print('캐시백 대상자 수:',len(LJ_cashback_target))

print('이벤트 참여율:',round(len(LJ_cashback_target)/Lotte_Jamsil_BuyCustomer*100, 2))

LJ_cashback_won = len(LJ_cashback_target) * 20000

print('캐시백 제공액:',LJ_cashback_won)

잠실점
캐시백 대상자 수: 6
이벤트 참여율: 46.15
캐시백 제공액: 120000


In [17]:
print('고객당 단가')

LM_Price_Sum = Lotte_Myeongdong_df['결제금액'].sum()

Price_Per_Customer = LM_Price_Sum / Lotte_Myeongdong_BuyCustomer

print('본점:',Price_Per_Customer)

LJ_Price_Sum = Lotte_Jamsil_df['결제금액'].sum()

Price_Per_Customer = LJ_Price_Sum / Lotte_Jamsil_BuyCustomer

print('잠실점:',Price_Per_Customer)


Total_Price_Sum = target_df['결제금액'].sum()

print('매출액')

print('본점:',LM_Price_Sum)
print('잠실점:',LJ_Price_Sum)
print('전체:',Total_Price_Sum)

고객당 단가
본점: 147855.6
잠실점: 123896.92307692308
매출액
본점: 7392780
잠실점: 1610660
전체: 9003440


In [18]:
print('구매건수 (명동/잠실/전체)')

LM_buy_cnt = len(Lotte_Myeongdong_df)
LJ_buy_cnt = len(Lotte_Jamsil_df)
Total_buy_cnt = len(target_df)
print(LM_buy_cnt, LJ_buy_cnt, Total_buy_cnt)


print('구매당 단가 (명동/잠실/전체)')

LM_price_per_buy = int(LM_Price_Sum / LM_buy_cnt)
LJ_price_per_buy = int(LJ_Price_Sum / LJ_buy_cnt)
Total_price_per_buy = int(Total_Price_Sum / Total_buy_cnt)

print(LM_price_per_buy, LJ_price_per_buy, Total_price_per_buy)

구매건수 (명동/잠실/전체)
83 35 118
구매당 단가 (명동/잠실/전체)
89069 46018 76300


In [19]:
LM_marketing_cost_per_customer = int(LM_cashback_won / Lotte_Myeongdong_BuyCustomer)

LJ_marketing_cost_per_customer = int(LJ_cashback_won / Lotte_Jamsil_BuyCustomer)

print('고객당 유치 비용')
print('본점', LM_marketing_cost_per_customer)
print('잠실점', LJ_marketing_cost_per_customer)


고객당 유치 비용
본점 10400
잠실점 9230


In [20]:
LM_marketing_cost_per_take = round(LM_cashback_won / LM_Price_Sum, 2) * 100

LJ_marketing_cost_per_take = round(LJ_cashback_won / LJ_Price_Sum, 3) * 100

print('매출액 대비 마케팅 비용 비율')
print('본점 {}%'.format(LM_marketing_cost_per_take))
print('잠실점 {}%'.format(LJ_marketing_cost_per_take))

매출액 대비 마케팅 비용 비율
본점 7.000000000000001%
잠실점 7.5%


In [21]:
# condition_4 = KNdf_iaurora['결제날짜'] < 20240422

# compare_df = KNdf_iaurora[condition_4]

In [22]:
# compare_df['결제금액'].sum()

In [23]:
iaurora = sa.open("레드테이블x아이오로라")
df_iaurora = iaurora.worksheet("외부인원_NEW_결제내역")
values = df_iaurora.get_all_values()
header, rows = values[0], values[1:]
df_iaurora = pd.DataFrame(rows, columns=header)

kona = sa.open("코나아이X코리아트래블카드")
df_kona = kona.worksheet("결제내역(코나)")
values = df_kona.get_all_values()
header, rows = values[1], values[2:]
df_kona = pd.DataFrame(rows, columns=header)

df_kona = df_kona[['id', '카드번호', '사업자번호', '결제날짜', '결제시간', '시/도', '시/군/구', '카테고리_대분류',
       'category_one', 'category_two', '상점명', '주소', '결제금액', '결제후금액','userId']]

df_iaurora = df_iaurora[['id','masking_card_no','16자리 카드번호',
       '사업자번호', '결제날짜', '결제시간', '시/도', '시/군/구', '카테고리_대분류', 'category_one',
       'category_two', '상점명', '주소', 'ktc_가맹여부', 'store_id', '결제금액',
       '결제후잔액', 'user_id']]

df_iaurora.rename(columns={'16자리 카드번호':'카드번호'}, inplace=True)

df_merge = pd.concat([df_kona, df_iaurora], axis=0)
df_merge = df_merge[df_merge['결제날짜'] != ' ']
df_merge['결제날짜'] = df_merge['결제날짜'].apply(lambda x:int(x))
df_merge['결제금액'] = df_merge['결제금액'].apply(lambda x:int(x))

df_merge.head()

,id,카드번호,사업자번호,결제날짜,결제시간,시/도,시/군/구,카테고리_대분류,category_one,category_two,상점명,주소,결제금액,결제후금액,userId,masking_card_no,ktc_가맹여부,store_id,결제후잔액,user_id
0,26,9410411100080104,1040675289,20240423,154643,서울특별시,중구,쇼핑,"생활,편의",편의점,GS25 소공점,서울특별시 중구 소공동 24,1500,250500,3101644687,NaN,NaN,NaN,NaN,NaN
1,28,9410411100080104,1048126067,20240423,155934,서울특별시,중구,쇼핑,"쇼핑,유통",백화점,롯데백화점 본점,서울특별시 중구 소공동 1,105000,145503,3101644687,NaN,NaN,NaN,NaN,NaN
2,50,9410411100080104,1048126067,20240423,161047,서울특별시,중구,쇼핑,"쇼핑,유통",백화점,롯데백화점 본점,서울특별시 중구 소공동 1,97300,48413,3101644687,NaN,NaN,NaN,NaN,NaN
3,55,9410411100080021,1048126067,20240424,125116,서울특별시,중구,쇼핑,"쇼핑,유통",백화점,롯데백화점 본점,서울특별시 중구 소공동 1,15000,14460,3101644671,NaN,NaN,NaN,NaN,NaN
4,57,9410411100080088,1048126067,20240424,125135,서울특별시,중구,쇼핑,"쇼핑,유통",백화점,롯데백화점 본점,서울특별시 중구 소공동 1,18000,282000,3101644832,NaN,NaN,NaN,NaN,NaN


In [24]:
condition_4 = (df_merge['결제날짜'] <= 20240419) & (df_merge['결제날짜'] >= 20240401)

df_compare = df_merge[condition_4]

condition_5 = (df_merge['결제날짜'] >= 20240422) & (df_merge['결제날짜'] <= 20240510)

df_compare_2 = df_merge[condition_5]

In [25]:
df_groupby_1 = df_compare.groupby(['결제날짜'])['결제금액'].sum().reset_index()

df_1_daily_avg_sales = int(df_groupby_1['결제금액'].mean())

df_groupby_2 = df_compare_2.groupby(['결제날짜'])['결제금액'].sum().reset_index()

df_2_daily_avg_sales = int(df_groupby_2['결제금액'].mean())

print('일 평균 매출')

print('프로모션 이전: {}'.format(df_1_daily_avg_sales))
print('프로모션 기간: {}'.format(df_2_daily_avg_sales))

일 평균 매출
프로모션 이전: 3378200
프로모션 기간: 5335518


In [26]:
non_c_list = target_df[target_df['16자리 카드번호'] == ""]['masking_card_no'].unique().tolist()

In [27]:
daily_avg_customer_cnt_1 = round(df_compare['카드번호'].nunique() / 21, 1)
daily_avg_customer_cnt_2 = round(df_compare_2['카드번호'].nunique() / 19, 1)

print('일 평균 구매 고객 수')

print('프로모션 이전: {}'.format(daily_avg_customer_cnt_1))
print('프로모션 기간: {}'.format(daily_avg_customer_cnt_2))

일 평균 구매 고객 수
프로모션 이전: 22.9
프로모션 기간: 32.5


In [28]:
df_compare_2['결제금액'].sum() / df_compare_2['카드번호'].nunique()

164302.8411669368

In [29]:
daily_price_per_customer_1 = df_1_daily_avg_sales / daily_avg_customer_cnt_1
daily_price_per_customer_2 = df_2_daily_avg_sales / daily_avg_customer_cnt_2

print('객단가')

print('프로모션 이전: {}'.format(daily_price_per_customer_1))
print('프로모션 기간: {}'.format(daily_price_per_customer_2))

객단가
프로모션 이전: 147519.65065502183
프로모션 기간: 164169.78461538462


In [30]:
Lc_list1 = target_df[target_df['16자리 카드번호']!='']['16자리 카드번호'].unique().tolist()
Lc_list2 = target_df[target_df['16자리 카드번호']=='']['masking_card_no'].unique().tolist()

In [31]:
df_Lotte_Customer = df_merge[(df_merge['masking_card_no'].isin(Lc_list2)) | (df_merge['카드번호'].isin(Lc_list1))]

df_NotLotte_Customer = df_merge[(~df_merge['masking_card_no'].isin(Lc_list2)) & (~df_merge['카드번호'].isin(Lc_list1))]

In [32]:
# # 롯데 고객 / 롯데 외 고객
# df_Lotte_Customer = df_merge[df_merge['카드번호'].isin(Lc_list)]
# df_NotLotte_Customer = df_merge[~df_merge['카드번호'].isin(Lc_list)]

In [33]:
LC_Category_CNT = df_Lotte_Customer['카테고리_대분류'].value_counts().reset_index()
LC_Category_SUM = df_Lotte_Customer.groupby('카테고리_대분류')['결제금액'].sum().reset_index()

df_LC_CATEGORY = pd.merge(LC_Category_CNT, LC_Category_SUM, how='inner', left_on = '카테고리_대분류', right_on = '카테고리_대분류')

df_LC_CATEGORY['구매건 단가'] = df_LC_CATEGORY['결제금액'] // df_LC_CATEGORY['count']

df_LC_CATEGORY['업종별 거래건수 비율'] = round(df_LC_CATEGORY['count'] / df_LC_CATEGORY['count'].sum() * 100, 2)
df_LC_CATEGORY['업종별 거래금액 비율'] = round(df_LC_CATEGORY['결제금액'] / df_LC_CATEGORY['결제금액'].sum() * 100, 2)

df_LC_CATEGORY

,카테고리_대분류,count,결제금액,구매건 단가,업종별 거래건수 비율,업종별 거래금액 비율
0,쇼핑,589,16836310,28584,51.22,62.03
1,식음료,416,7255430,17440,36.17,26.73
2,현지교통,46,767180,16677,4.00,2.83
3,문화/오락,41,460000,11219,3.57,1.69
4,기타,39,626120,16054,3.39,2.31
5,의료/뷰티,15,1022900,68193,1.30,3.77
6,카테고리 추가필요,4,176400,44100,0.35,0.65


In [34]:
# 쇼핑에서 편의점 제외시
17711340 / (854 - len(df_Lotte_Customer[df_Lotte_Customer['category_two'] == '편의점']))

28474.8231511254

In [35]:
NLC_Category_CNT = df_NotLotte_Customer['카테고리_대분류'].value_counts().reset_index()
NLC_Category_SUM = df_NotLotte_Customer.groupby('카테고리_대분류')['결제금액'].sum().reset_index()

df_NLC_CATEGORY = pd.merge(NLC_Category_CNT, NLC_Category_SUM, how='inner', left_on = '카테고리_대분류', right_on = '카테고리_대분류')

df_NLC_CATEGORY['구매건 단가'] = df_NLC_CATEGORY['결제금액'] // df_NLC_CATEGORY['count']

df_NLC_CATEGORY['업종별 거래건수 비율'] = round(df_NLC_CATEGORY['count'] / df_NLC_CATEGORY['count'].sum() * 100,2)
df_NLC_CATEGORY['업종별 거래금액 비율'] = round(df_NLC_CATEGORY['결제금액'] / df_NLC_CATEGORY['결제금액'].sum() * 100,2)

df_NLC_CATEGORY

,카테고리_대분류,count,결제금액,구매건 단가,업종별 거래건수 비율,업종별 거래금액 비율
0,쇼핑,22563,259426514,11497,55.23,48.72
1,식음료,13280,191153651,14394,32.50,35.90
2,기타,1905,23797030,12491,4.66,4.47
3,현지교통,1642,24934460,15185,4.02,4.68
4,문화/오락,931,15962550,17145,2.28,3.00
5,의료/뷰티,474,16073154,33909,1.16,3.02
6,카테고리 추가필요,61,1159802,19013,0.15,0.22


In [36]:
# 쇼핑에서 편의점 제외시
df_NotLotte_Customer[(df_NotLotte_Customer['카테고리_대분류'] == '쇼핑') &(df_NotLotte_Customer['category_two'] != '편의점')]['결제금액'].sum() / (22162 - len(df_NotLotte_Customer[df_NotLotte_Customer['category_two'] == '편의점']))

19261.012287606798